This notebook models stock data that has been pre-processed with news sentiment data, then predicts the Closing, High, and Low values for the current buisness day and following business day. Below is the process for preparing the data then modeling the data for the best predictions.